# AIPitBoss: OpenAI API Integration Tutorial

This notebook demonstrates how to:
1. Obtain an OpenAI API key
2. Configure AIPitBoss to use your API key
3. Create a simple question-answering application using AIPitBoss and OpenAI

## Step 1: Getting an OpenAI API Key

To use OpenAI's API with AIPitBoss, you need an API key. Here's how to get one:

1. **Create an OpenAI account**: Go to [OpenAI's website](https://openai.com/) and sign up for an account if you don't already have one.

2. **Navigate to the API section**: After logging in, go to the [API keys page](https://platform.openai.com/api-keys).

3. **Create a new API key**: Click on "Create new secret key" and give it a name (e.g., "AIPitBoss Integration").

4. **Save your API key**: OpenAI will show you the API key once. Copy it and store it securely as you won't be able to see it again.

5. **Set up billing**: Make sure you have billing set up in your OpenAI account to use the API.

![OpenAI API Key Page](https://platform.openai.com/docs/images/api-keys-page.webp)

**Important**: Your API key is a secret. Never share it publicly or commit it to a public repository.

## Step 2: Configuring AIPitBoss with Your API Key

AIPitBoss offers three ways to provide your API key:

1. **Directly in code** (not recommended for production)
2. **Using environment variables**
3. **Using a .keys.json file** (recommended for development)

Let's explore each method:

### Method 1: Direct API Key (For Testing Only)

In [ ]:
# Import AIPitBoss
from aipitboss import OpenAIService

# Initialize with direct API key (not recommended for shared or production code)
# openai = OpenAIService(api_key='your-api-key-here')

# Note: We're not executing this code as it's not the recommended approach

### Method 2: Using Environment Variables

In [ ]:
import os
from aipitboss import OpenAIService

# Set environment variable (in a real environment, you'd set this in your system)
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# Initialize service using environment variable
# openai = OpenAIService(use_env=True)

# Check if the environment variable is already set
if "OPENAI_API_KEY" in os.environ:
    print("OPENAI_API_KEY environment variable is set")
else:
    print("OPENAI_API_KEY environment variable is not set")

### Method 3: Using a Keys File (Recommended for Development)

In [ ]:
import json
from pathlib import Path
from aipitboss import KeyManager

# Path to the keys file
keys_file = ".keys.json"

# Check if keys file exists
if Path(keys_file).exists():
    print(f"Keys file exists at {Path(keys_file).absolute()}")
    
    # Load and print available services (without showing the actual keys)
    with open(keys_file, 'r') as f:
        keys = json.load(f)
    print(f"Available keys for services: {list(keys.keys())}")
else:
    print("Keys file doesn't exist. Creating one...")
    
    # Demonstration of how to create a keys file
    # Replace 'your-api-key-here' with your actual API key
    # KeyManager.save_keys({'openai': 'your-api-key-here'}, keys_file)
    # print(f"Keys file created at {Path(keys_file).absolute()}")

## Step 3: Using AIPitBoss to Interact with OpenAI

Now that we have our API key configured, let's use AIPitBoss to interact with OpenAI's API:

In [ ]:
from aipitboss import OpenAIService

# Initialize OpenAI service using the keys file
openai = OpenAIService(keys_file=".keys.json")

# Print confirmation (without revealing the actual key)
if openai.api_key:
    print("✓ Successfully initialized OpenAI service with API key")
    print(f"  API key starts with: {openai.api_key[:8]}...")
else:
    print("✗ Failed to initialize OpenAI service with API key")

### Creating a Simple Question-Answering Application

Let's create a simple function to ask questions to OpenAI's models using our AIPitBoss integration:

In [ ]:
def ask_question(question, model="gpt-3.5-turbo"):
    """Ask a question to OpenAI and get the response."""
    # Prepare the messages
    messages = [
        {"role": "system", "content": "You are a helpful, concise assistant."},
        {"role": "user", "content": question}
    ]
    
    # Make the API call
    response = openai.chat_completion(
        messages=messages,
        model=model,
        temperature=0.7,
        max_tokens=150
    )
    
    # Extract the answer text
    answer = response["choices"][0]["message"]["content"]
    return answer

Now let's try asking some questions:

In [ ]:
# Ask a question
question = "What is artificial intelligence in simple terms?"
print(f"Question: {question}")
print("\nAnswer:")
answer = ask_question(question)
print(answer)

In [ ]:
# Ask another question
question = "What are the three main types of machine learning?"
print(f"Question: {question}")
print("\nAnswer:")
answer = ask_question(question)
print(answer)

### Interactive Question Answering

Let's create a simple interactive interface to ask questions:

In [ ]:
# Only run this in a notebook environment
from IPython.display import display, clear_output
import ipywidgets as widgets

# Create text input for questions
question_input = widgets.Text(
    value='',
    placeholder='Type your question here',
    description='Question:',
    disabled=False,
    layout=widgets.Layout(width='80%')
)

# Create output area for answers
output = widgets.Output()

# Function to handle submission
def on_submit(b):
    with output:
        clear_output()
        question = question_input.value
        if not question.strip():
            print("Please enter a question.")
            return
            
        print(f"Q: {question}")
        print("\nThinking...")
        
        try:
            answer = ask_question(question)
            clear_output()
            print(f"Q: {question}\n")
            print(f"A: {answer}")
        except Exception as e:
            clear_output()
            print(f"Error: {e}")

# Create button
button = widgets.Button(
    description='Ask',
    disabled=False,
    button_style='info',
    tooltip='Ask question',
    icon='question'
)
button.on_click(on_submit)

# Also trigger on Enter key
question_input.on_submit(lambda sender: on_submit(None))

# Display the interface
display(widgets.HBox([question_input, button]))
display(output)

## Additional Features

### Streaming Responses

AIPitBoss also supports streaming responses from OpenAI. Here's how to use it:

In [ ]:
import requests
from aipitboss import StreamProcessor

def ask_question_streaming(question, model="gpt-3.5-turbo"):
    """Ask a question and stream the response."""
    # Prepare URL and headers
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {openai.api_key}",
        "Content-Type": "application/json"
    }
    
    # Prepare data
    data = {
        "model": model,
        "messages": [
            {"role": "system", "content": "You are a helpful, concise assistant."},
            {"role": "user", "content": question}
        ],
        "stream": True  # Enable streaming
    }
    
    print(f"Q: {question}\n")
    print("A: ", end="", flush=True)
    
    # Make streaming request
    with requests.post(url, json=data, headers=headers, stream=True) as response:
        response.raise_for_status()
        
        # Process the stream
        return StreamProcessor.process_openai_stream(
            response.iter_lines(),
            chunk_handler=StreamProcessor.print_stream
        )

In [ ]:
# Try the streaming function
full_response = ask_question_streaming("Write a short poem about artificial intelligence.")

## Conclusion

You've successfully learned how to:
1. Obtain an OpenAI API key
2. Configure AIPitBoss to use your API key
3. Use AIPitBoss to create a question-answering application
4. Implement streaming responses

AIPitBoss provides a simple, intuitive interface to work with OpenAI's API, making it easy to build AI-powered applications. You can extend this notebook to explore more features like image generation, model fine-tuning, and more.

For more examples, check out the other examples in the `examples/` directory of the AIPitBoss package.